In [ ]:
import copy

# Current Board Class

In [ ]:
class CurrentBoard:
  def __init__(self):
    # Initialize the game board with empty spaces
    self.board = [[' ' for _ in range(5)] for _ in range(3)]
    # Set initial positions for the hare and hounds
    self.board[1][4] = 'H'
    self.board[0][1] = 'A'
    self.board[1][0] = 'B'
    self.board[2][1] = 'C'
    # Determine the current state of the game
    self.state = self.state_of_board()
    # Dictionary to map move indices to positions
    self.move_index_mapping = {}

  # Display the current state of the board
  def display(self, game_display=False, player_piece = None):
    for i in range(3):
            for j in range(5):
                if self.board[i][j] not in ['H', 'A', 'B', 'C']:
                    self.board[i][j] = ' '
    if game_display:
      possible_moves = self.all_possible_moves(player_piece)
      self.move_index_mapping = {}
      ind = 1
      for move in possible_moves:
          x, y = move
          self.move_index_mapping[ind] = (x, y)
          self.board[x][y] = str(ind)
          ind += 1
    print(self.board[0][0] + "  " + self.board[0][1] + "--" + self.board[0][2] + "--" + self.board[0][3] + "  " + self.board[0][4])
    print(" / | \\|/ | \\ ")
    print(self.board[1][0] + "--" + self.board[1][1] + "--" + self.board[1][2] + "--" + self.board[1][3] + "--" + self.board[1][4])
    print(" \\ | /|\\ | / ")
    print(self.board[2][0] + "  " + self.board[2][1] + "--" + self.board[2][2] + "--" + self.board[2][3] + "  " + self.board[2][4])

  # Return the move index mapping
  def index_mapping(self):
    return self.move_index_mapping

  # Move the hare to a new position
  def move_hare(self, x, y):
    for i in range(len(self.board)):
      for j in range(len(self.board[i])):
          if self.board[i][j] == 'H':
              self.board[i][j] = ' '
              break
    self.board[x][y] = 'H'
    return True

  # Move a hound to a new position
  def move_hound(self, hound_symbol, x, y):
    for i in range(len(self.board)):
        for j in range(len(self.board[i])):
            if self.board[i][j] == hound_symbol:
                self.board[i][j] = ' '
                break
    self.board[x][y] = hound_symbol

    return True

  # Get the position of a player piece on the board
  def get_position(self, player_piece):
    for i in range(len(self.board)):
      for j in range(len(self.board[i])):
        if self.board[i][j] == player_piece:
          position = (i, j)
    return position

  # Get the positions of all hounds on the board
  def get_hound_positions(self):
    hound_positions = []
    for i in range(len(self.board)):
      for j in range(len(self.board[i])):
        if self.board[i][j] in ['A', 'B', 'C']:
          hound_positions.append((i, j))
    return hound_positions

  # Determine illegal moves based on the current position
  def get_illegal_moves(self, current_position):
    illegal_moves = [(0, 0), (0, 4), (2, 0), (2, 4)]

    if current_position == (0, 2) or current_position == (2, 2):
      illegal_moves.extend([(1, 1), (1, 3)])

    elif current_position == (1,1) or current_position == (1,3):
      illegal_moves.extend([(0, 2), (2, 2)])

    return illegal_moves

  # Switch player pieces
  def other(self, piece):
    if piece == "X":
        return "H"
    return "X"

  # Generate all possible moves for a player piece
  def all_possible_moves(self, player_piece):
    possible_moves = []
    for i in range(len(self.board)):
      for j in range(len(self.board[i])):
          if self.board[i][j] == player_piece:
            current_position = (i,j)
            if player_piece == "H":
                moves = [
                    (i - 1, j), (i + 1, j),  # Sideways
                    (i, j - 1), (i, j + 1),  # Forward and Backwards
                    (i - 1, j - 1), (i - 1, j + 1),  # Diagonally
                    (i + 1, j - 1), (i + 1, j + 1)  # Diagonally
                ]
            else:
                moves = [
                    (i, j + 1),  # Forward
                    (i + 1, j), (i -1, j), # Sideways
                    (i - 1, j + 1), (i + 1, j + 1)  # Diagonally
                ]

            valid_moves = [(x, y) for (x, y) in moves if 0 <= x < len(self.board) and 0 <= y < len(self.board[0]) and (x, y) not in self.get_illegal_moves(current_position) and self.board[x][y] == " "]
            possible_moves.extend(valid_moves)

    return possible_moves

  # Generate all possible moves for hounds
  def all_hound_moves(self):
    hound_moves = []

    hound_moves.extend(self.all_possible_moves("A"))
    hound_moves.extend(self.all_possible_moves("B"))
    hound_moves.extend(self.all_possible_moves("C"))

    return hound_moves

  # Check if hounds are in a line
  def hounds_in_line(self, p1, p2, p3):
    if p1[1] == p2[1] == p3[1]:
        return 3
    elif p1[1] == p2[1] or p1[1] == p3[1] or p2[1] == p3[1]:
        return 2
    else:
        return 1

  # Create a deep copy of the current board
  def get_board_copy(self):
    return copy.deepcopy(self.board)

  # Determine the current state of the board
  def state_of_board(self):
    hare_position = self.get_position("H")
    hound_positions = self.get_hound_positions()

    hare_to_left = all(hare_position[1] <= hound_position[1] for hound_position in hound_positions)

    if hare_position and hare_to_left:
        return "H"

    elif not self.all_possible_moves("H"):
      return "X"

    return "U"

  # Check if the hare is boxed in by hounds
  def hare_boxed_in(self):
    hare_position = self.get_position("H")
    h1, h2, h3 = self.get_hound_positions()

    i, j = hare_position

    boxed_in_top_left = [(i,j-1), (i-1, j-1), (i-1, j)]
    boxed_in_bottom_left = [(i, j-1), (i+1, j-1), (i+1, j)]

    if all(position in [h1, h2, h3] for position in boxed_in_top_left):
      if i == 2:
        return True
    elif all(position in [h1, h2, h3] for position in boxed_in_bottom_left):
      if i == 0:
        return True
    else:
        return False

    return False

  # Evaluate the current board and calculate its score
  def evaluate_board(self):
    score = 0
    hare_position = self.get_position("H")
    hound_positions = self.get_hound_positions()
    h1, h2, h3 = self.get_hound_positions()
    MIDDLE_BOARD = (1, 2)
    HARE_START = (1, 4)

    # Check if hare has reached the middle piece
    if hare_position == MIDDLE_BOARD:
        score += 5000

    # Calculate the number of moves a hare can make
    all_hare_moves = self.all_possible_moves("H")

    forward_hare_moves = 0
    sideways_hare_moves = 0
    backwards_hare_moves = 0

    hare_x, hare_y = hare_position
    forward_hare_positions = []
    sideways_hare_positions = []

    for move in all_hare_moves:
        # Forward moves (y < hare position)
        if move[1] < hare_position[1]:
            forward_hare_positions.append(move)
            forward_hare_moves += 1

        # Sideways moves (y == hare position)
        elif move[1] == hare_position[1]:
            sideways_hare_positions.append(move)
            sideways_hare_moves += 1

        # Backwards moves (y > hare position)
        elif move[1] > hare_position[1]:
            backwards_hare_moves += 1

    score += (forward_hare_moves * 3) #Forward moves worth more
    score += sideways_hare_moves
    score -= (backwards_hare_moves * 2) #Backwards moves are in favour of hounds

    # Calculate the number of forward moves hounds can make
    forward_hound_moves = 0
    forward_hound_positions = []
    sideways_hound_moves = 0

    for hound in ["A", "B", "C"]:
        all_hound_moves = self.all_possible_moves(hound)
        hound_position = self.get_position(hound)

        for move in all_hound_moves:
            # Forward moves (y > hound position)
            if move[1] > hound_position[1]:
                forward_hound_positions.append(move)
                forward_hound_moves += 1

            # Sideways moves (y == hound position)
            elif move[1] == hound_position[1]:
                sideways_hound_moves += 1


            if move in forward_hare_positions:
                score -= 300
            elif move in sideways_hare_positions:
                score -= 150

    score -= forward_hound_moves

    # If hound can move into a forward hare position
    for fhm in forward_hound_positions:
      if(fhm in forward_hare_positions):
        score -= 250


    # Check if the hounds are in line
    num_hounds_in_line = self.hounds_in_line(h1, h2, h3)
    if num_hounds_in_line == 3:
      score -= 600
    elif self.hounds_in_line == 2:
      score -= 200
    else:
      score += 200

    # Check if hound reached the middle piece
    if MIDDLE_BOARD in [h1, h2, h3]:
        score -= 5000

    #Check if a hound has gone to the end piece & cannot be moved again
    if HARE_START in [h1, h2, h3]:
      score += 3000

    if (self.hare_boxed_in()):
      score -= 800
    else:
      score += 150

    # Hares only move is back to the start
    if (len(all_hare_moves) == 1 and all_hare_moves[0] == HARE_START):
      score -= 4000

    # If hare will have no moves
    if (self.all_possible_moves("H") is None):
      score -10000
    return score

# Search Tree Node Class

In [ ]:
class SearchTreeNode:
  def __init__(self, board_instance, playing_as, ply=0, max_ply=3):
    self.children = []
    self.value_is_assigned = False
    self.ply_depth = ply
    self.max_ply = max_ply
    self.current_board = board_instance
    self.move_for = playing_as
    self.value = self.current_board.evaluate_board()

    # Check if the game is unfinished and maximum depth is not reached
    if self.current_board.state == "U":
      if self.ply_depth < self.max_ply:
        # Generate child nodes if games unfinished
          self.generate_children()
      else:
        # Assign board value if maximum depth is reached
          self.value = self.current_board.evaluate_board()
          self.value_is_assigned = True
    else:
      # If the game is finished
      # Computers Move
      if ((self.ply_depth % 2) == 0):
        if(self.move_for == self.current_board.state_of_board()):
          self.value = -10000
        else:
          self.value = 10000

      # Players Move
      else:
        if(self.move_for == self.current_board.state_of_board()):
          self.value = 10000
        else:
          self.value = -10000

      self.value_is_assigned = True

  def minimax(self):
    # If the node's value is already assigned, return it
    if self.value_is_assigned:
      return self.value

    # Sort children nodes based on their minimax values
    self.children = sorted(self.children, key = lambda x:x.minimax(), reverse = True)

    if ((self.ply_depth % 2) == 0):
      # computers move
      if self.children:
            self.value = self.children[-1].value  # Smallest value for computer
      else:
            self.value = -10000
    else:
      #players move
      if self.children:
          self.value = self.children[0].value  # Highest value for player
      else:
          self.value = 10000

    self.value_is_assigned = True

    return self.value

  def generate_children(self):
    # Generate child nodes based on the current player's move
    if self.move_for == "H":
      for move in self.current_board.all_possible_moves(self.move_for):
        # Create a copy of the current board
          board_for_next_move = CurrentBoard()
          board_for_next_move.board = self.current_board.get_board_copy()
          x, y = move
          # Apply hare's move to the new board
          board_for_next_move.move_hare(x, y)
          # Create a child node with the new board and switch player
          self.children.append(SearchTreeNode(board_for_next_move, self.current_board.other(self.move_for), ply=self.ply_depth + 1, max_ply=self.max_ply))
    elif self.move_for == "X":
      for hound_symbol in ["A", "B", "C"]:
        for move in self.current_board.all_possible_moves(hound_symbol):
          # Create a copy of the current board
          board_for_next_move = CurrentBoard()
          board_for_next_move.board = self.current_board.get_board_copy()
          x, y = move
          # Apply hound's move to the new board
          board_for_next_move.move_hound(hound_symbol, x, y)
          # Create a child node with the new board and switch player
          self.children.append(SearchTreeNode(board_for_next_move, self.current_board.other(self.move_for), ply=self.ply_depth + 1, max_ply=self.max_ply))

#Play Game Class

In [ ]:
def play_Hare_and_Hounds():
  # Prompt the user to choose a piece to play as
  response = input("Do you wish to play Hare (H) or Hound (X)?")
  # Create a new instance of the CurrentBoard class
  cb = CurrentBoard()
  # Determine which piece the player is playing as
  player_is_playing = cb.other(cb.other(response))

  # Set the initial turn based on the player's piece
  if player_is_playing == "H":
    players_turn = True
  else:
    players_turn = False

   # Main game loop
  for _ in range(21): #21 Iterations. Handles the lack of a method to implement "Stalling" by hounds.
    if players_turn:
      if player_is_playing == "H":
        # Display the board highlighting possible moves for the hare
        cb.display(game_display=True, player_piece = player_is_playing)

      else:
        # Display the board and prompt the player to select a hound
        cb.display()
        hound = input("Select your hound (A, B, C):").upper()
        cb.display(game_display=True, player_piece = hound)

      # Prompt the player to make a move
      choice = input("Make your move ")
      ind = int(choice)
      index_map = cb.index_mapping()
      x, y = index_map[ind]

      # Execute the selected move
      if player_is_playing == "H":
        cb.move_hare(x, y)
      else:
        cb.move_hound(hound, x, y)

      # Display the updated board
      cb.display()

    else:
      # Computer's turn
      # Create a search tree node and apply minimax algorithm
      search_tree = SearchTreeNode(cb, cb.other(player_is_playing))
      search_tree.minimax()
      # Update the current board with the computer's move
      cb = search_tree.children[-1].current_board

    # Check the game state after each turn
    if cb.state_of_board() != "U":
      if cb.state_of_board() == "H":
        print( "Hare Wins")

      elif cb.state_of_board() == "X":
        print( "Hounds Win")
        cb.display()
      break

    # Switch turns
    players_turn = not players_turn

  # Handle stalemate
  if cb.state_of_board() == "U":
        print("Stalemate - Hare Wins")

# Play Game

In [ ]:
play_Hare_and_Hounds()

Do you wish to play Hare (H) or Hound (X)?H
Player piece: H
Players turn: True
   A-- --2   
 / | \|/ | \ 
B-- -- --1--H
 \ | /|\ | / 
   C-- --3   
Make your move 1
   A-- --    
 / | \|/ | \ 
B-- -- --H-- 
 \ | /|\ | / 
   C-- --    
   A-- --1   
 / | \|/ | \ 
B-- --C--H--3
 \ | /|\ | / 
    -- --2   
Make your move 2
   A-- --    
 / | \|/ | \ 
B-- --C-- -- 
 \ | /|\ | / 
    -- --H   
   A-- --    
 / | \|/ | \ 
 --B--C--1--3
 \ | /|\ | / 
    --2--H   
Make your move 2
   A-- --    
 / | \|/ | \ 
 --B--C-- -- 
 \ | /|\ | / 
    --H--    
   A-- --    
 / | \|/ | \ 
 -- --C-- -- 
 \ | /|\ | / 
   B--H--1   
Make your move 1
   A-- --    
 / | \|/ | \ 
 -- --C-- -- 
 \ | /|\ | / 
   B-- --H   
   A-- --    
 / | \|/ | \ 
 -- --C--1--2
 \ | /|\ | / 
    --B--H   
Make your move 1
   A-- --    
 / | \|/ | \ 
 -- --C--H-- 
 \ | /|\ | / 
    --B--    
   A-- --1   
 / | \|/ | \ 
 -- --C--H--2
 \ | /|\ | / 
    -- --B   
Make your move 1
   A-- --H   
 / | \|/ | \ 
 -- --C-- -- 
 \ | /|

KeyboardInterrupt: Interrupted by user

#Testing

## Display Board

In [ ]:
cb = CurrentBoard()

In [ ]:
cb.display()

   A-- --    
 / | \|/ | \ 
B-- -- -- --H
 \ | /|\ | / 
   C-- --    


In [ ]:
cb.possible_forward_moves("H")

3

In [ ]:
cb.evaluate_board()

-2

##Check Moves

In [ ]:
cb.all_possible_moves("H")

[(1, 3), (0, 3), (2, 3)]

In [ ]:
cb.all_possible_moves("A")

[(0, 2), (1, 1), (1, 2)]

In [ ]:
cb.all_possible_moves("B")

[(1, 1)]

In [ ]:
cb.all_possible_moves("C")

[(2, 2), (1, 1), (1, 2)]

In [ ]:
cb.all_hound_moves()

[(0, 2), (1, 1), (1, 2), (1, 1), (2, 2), (1, 1), (1, 2)]

In [ ]:
cb.get_hound_positions()

[(0, 1), (1, 0), (2, 1)]

In [ ]:
test_board = CurrentBoard()
for i in range(len(test_board.board)):
    for j in range(len(test_board.board[i])):
        if test_board.board[i][j] in ['H', 'A', 'B', 'C']:
            test_board.board[i][j] = ' '

test_board.board[0][2] = 'H'
test_board.board[2][2] = 'A'
test_board.board[1][4] = 'B'
test_board.board[0][3] = 'C'

test_board.display()

    --H--C   
 / | \|/ | \ 
 -- -- -- --B
 \ | /|\ | / 
    --A--    


In [ ]:
test_board.all_forward_hound_moves()

1

In [ ]:
test_board.all_possible_moves("H")

[(1, 2), (0, 1)]

In [ ]:
test_board.all_possible_moves("A")

[(2, 3), (1, 2)]

## Testing State of Board

### Hare Wins

In [ ]:
test_board = CurrentBoard()
for i in range(len(test_board.board)):
    for j in range(len(test_board.board[i])):
        if test_board.board[i][j] in ['H', 'A', 'B', 'C']:
            test_board.board[i][j] = ' '

test_board.board[1][1] = 'H'
test_board.board[2][2] = 'A'
test_board.board[1][4] = 'B'
test_board.board[0][3] = 'C'

test_board.display()

    -- --C   
 / | \|/ | \ 
 --H-- -- --B
 \ | /|\ | / 
    --A--    


In [ ]:
test_board.state_of_board()

'H'

In [ ]:
test_st = SearchTreeNode(test_board, "H")

In [ ]:
test_board.possible_forward_moves("H")

1

In [ ]:
test_st.children

In [ ]:
test_st.children[0].children[0].current_board.display()

   H-- --C   
 / | \|/ | \ 
 -- -- -- --B
 \ | /|\ | / 
    -- --A   


### Hounds Win

In [ ]:
test_board = CurrentBoard()
for i in range(len(test_board.board)):
    for j in range(len(test_board.board[i])):
        if test_board.board[i][j] in ['H', 'A', 'B', 'C']:
            test_board.board[i][j] = ' '

test_board.board[1][4] = 'H'
test_board.board[2][3] = 'A'
test_board.board[1][2] = 'B'
test_board.board[0][3] = 'C'

test_board.display()

    -- --C   
 / | \|/ | \ 
 -- --B-- --H
 \ | /|\ | / 
    -- --A   


In [ ]:
test_board.state_of_board()

'U'

## Search Tree

In [ ]:
test_st = SearchTreeNode(test_board, 'X')

In [ ]:
test_st.children

In [ ]:
test_st.children[1].current_board.display()

    -- --C   
 / | \|/ | \ 
 -- -- --B--H
 \ | /|\ | / 
    -- --A   


In [ ]:
st = SearchTreeNode(cb, 'H')

In [ ]:
st.minimax()

-3

In [ ]:
st.current_board.evaluate_board()

-2

In [ ]:
st.children[1].children[3].children[2].children[1].value

749

In [ ]:
st.children[1].children[3].children[2].children[1].current_board.display()

   A-- --    
 / | \|/ | \ 
 --B--H-- -- 
 \ | /|\ | / 
    --C--    


In [ ]:
st.children[1].children

In [ ]:
st.children[1].children[2].value

-3

In [ ]:
st.children[1].children[3].value

-3

In [ ]:
st.children[1].children[3].children[2].children[1].current_board.display()

   A-- --    
 / | \|/ | \ 
 --B--H-- -- 
 \ | /|\ | / 
    --C--    


In [ ]:
st.children[0].children[4].current_board.display()

   A-- --    
 / | \|/ | \ 
B-- -- --H-- 
 \ | /|\ | / 
    --C--    


In [ ]:
st.children[0].children[4].children[0].children[0].current_board.display()

    --A--H   
 / | \|/ | \ 
B-- -- -- -- 
 \ | /|\ | / 
    --C--    
